# NASA SMD Question Answering Finetuning

This notebook demonstrates how to finetune a transformer model for extractive question answering using the NASA SMD QA benchmark dataset. We'll use the Hugging Face transformers library to fine-tune a pretrained language model on this task.

## Install Required Libraries

In [ ]:
!pip install transformers datasets accelerate evaluate scikit-learn matplotlib

## Import Libraries

In [ ]:
import numpy as np
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForQuestionAnswering,
    TrainingArguments,
    Trainer,
    default_data_collator,
    EvalPrediction
)
import evaluate
import collections
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

## Check for GPU

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## Load the NASA SMD QA Benchmark Dataset

This dataset is in SQuAD v2 format, which includes questions with and without answers.

In [ ]:
# Load the dataset
dataset = load_dataset("nasa-impact/nasa-smd-qa-benchmark")
print(f"Dataset loaded: {dataset}")

In [ ]:
# Explore the dataset structure
print("Dataset structure:")
print(dataset["train"][0])

## Initialize the Model and Tokenizer

We'll use a pretrained model as our starting point for finetuning.

In [ ]:
# Initialize the tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

print(f"Model and tokenizer initialized from {model_name}")

## Preprocess the Dataset

For question answering tasks, we need to:
1. Tokenize both the question and context
2. Find start and end positions of answers in the tokenized context
3. Handle no-answer cases (SQuAD v2)

In [ ]:
# Maximum sequence length for the model
max_length = 384
# Maximum length of the answer that can be generated
max_answer_length = 30
# Stride to use when splitting up a long context
doc_stride = 128
# Set to True if we have unanswerable questions (SQuAD v2)
squad_v2 = True

# Function to prepare the dataset features
def prepare_train_features(examples):
    # Some of the questions have lots of whitespace on the left, which is not useful and will make the
    # truncation of the context fail (the tokenized question will take a lots of space).
    # So we remove that left whitespace
    examples["question"] = [q.lstrip() for q in examples["question"]]

    # Tokenize our examples with truncation and padding, but keep the overflows using a stride.
    # This results in one example possibly giving several features when a context is long.
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

# Similar function for validation data, but with additional answer processing
def prepare_validation_features(examples):
    examples["question"] = [q.lstrip() for q in examples["question"]]

    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
    # corresponding example_id and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [ ]:
# Apply the preprocessing functions
tokenized_train_dataset = dataset["train"].map(
    prepare_train_features,
    batched=True,
    remove_columns=dataset["train"].column_names,
)

tokenized_validation_dataset = dataset["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=dataset["validation"].column_names,
)

print(f"Tokenized train dataset: {tokenized_train_dataset}")
print(f"Tokenized validation dataset: {tokenized_validation_dataset}")

## Evaluation Metrics

For question answering, we need to compute the exact match and F1 score between the predicted answers and the reference answers.

In [ ]:
# The metric to use for evaluation
metric = evaluate.load("squad_v2" if squad_v2 else "squad")

# Function to post process the results and compute metrics
def post_processing_function(examples, features, predictions, stage="eval"):
    # Post-processing: we match the start logits and end logits to answers in the original context.
    predictions = postprocess_qa_predictions(
        examples=examples,
        features=features,
        predictions=predictions,
        version_2_with_negative=squad_v2,
        n_best_size=20,
        max_answer_length=max_answer_length,
        null_score_diff_threshold=0.0,
        output_dir=None,
        prefix=stage,
    )
    
    # Format the result to the format the metric expects.
    if squad_v2:
        formatted_predictions = [
            {"id": k, "prediction_text": v, "no_answer_probability": 0.0} for k, v in predictions.items()
        ]
    else:
        formatted_predictions = [{"id": k, "prediction_text": v} for k, v in predictions.items()]

    references = [{"id": ex["id"], "answers": ex["answers"]} for ex in examples]
    return EvalPrediction(predictions=formatted_predictions, label_ids=references)

def compute_metrics(p: EvalPrediction):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

# Post-processing function for question answering predictions
def postprocess_qa_predictions(
    examples,
    features,
    predictions,
    version_2_with_negative=False,
    n_best_size=20,
    max_answer_length=30,
    null_score_diff_threshold=0.0,
    output_dir=None,
    prefix=None,
):
    if len(predictions) != 2:
        raise ValueError("predictions should be of size 2, but is {}".format(len(predictions)))
    all_start_logits, all_end_logits = predictions

    # Build a map from example to its features
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill
    all_predictions = collections.OrderedDict()

    # Let's loop over all the examples
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example
        feature_indices = features_per_example[example_index]

        min_null_prediction = None
        prelim_predictions = []

        # Looping through all the features associated to the current example
        for feature_index in feature_indices:
            # We need to get the token mappings from the indices to the text
            current_feature = features[feature_index]
            offset_mapping = current_feature["offset_mapping"]

            # First feature has the CLS index
            cls_index = current_feature["input_ids"].index(tokenizer.cls_token_id)

            # Get the prediction for start and end positions
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]

            # Get the score for the "no answer" option
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_prediction is None or min_null_prediction["score"] > feature_null_score:
                min_null_prediction = {
                    "offsets": (0, 0),
                    "score": feature_null_score,
                    "start_logit": start_logits[cls_index],
                    "end_logit": end_logits[cls_index],
                }

            # Go through all possible answer spans and score them
            start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if end_index < start_index or end_index - start_index + 1 > max_answer_length:
                        continue

                    prelim_predictions.append(
                        {
                            "offsets": (offset_mapping[start_index][0], offset_mapping[end_index][1]),
                            "score": start_logits[start_index] + end_logits[end_index],
                            "start_logit": start_logits[start_index],
                            "end_logit": end_logits[end_index],
                        }
                    )

        # If we don't have any valid predictions, just return the empty string
        if len(prelim_predictions) == 0:
            all_predictions[example["id"]] = ""
        else:
            # Only keep the best `n_best_size` predictions
            predictions = sorted(prelim_predictions, key=lambda x: x["score"], reverse=True)[:n_best_size]

            # Score differential for "no answer" prediction
            if version_2_with_negative:
                if min_null_prediction is not None:
                    # Predict "no answer" only if its score is the best by a margin over other answers
                    score_diff = min_null_prediction["score"] - predictions[0]["score"]
                    if score_diff > null_score_diff_threshold:
                        all_predictions[example["id"]] = ""
                    else:
                        all_predictions[example["id"]] = example["context"][predictions[0]["offsets"][0]:predictions[0]["offsets"][1]]
                else:
                    all_predictions[example["id"]] = example["context"][predictions[0]["offsets"][0]:predictions[0]["offsets"][1]]
            else:
                all_predictions[example["id"]] = example["context"][predictions[0]["offsets"][0]:predictions[0]["offsets"][1]]

    return all_predictions

## Configure Training Parameters

In [ ]:
# Set training parameters
training_args = TrainingArguments(
    output_dir="./nasa-smd-qa-model",
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    save_strategy="epoch",
    save_total_limit=2,
)

## Initialize Trainer and Start Training

In [ ]:
# Initialize our Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_validation_dataset,
    tokenizer=tokenizer,
    data_collator=default_data_collator,
    post_process_function=lambda examples, features, predictions: post_processing_function(dataset["validation"], features, predictions),
    compute_metrics=compute_metrics,
)

In [ ]:
# Start training
print("Starting training...")
trainer.train()

## Evaluate the Model

In [ ]:
# Evaluate the model
print("Evaluating the model...")
result = trainer.evaluate()
print(result)

## Test Model with Sample Questions

In [ ]:
# Function to get answer from the model for a given question and context
def get_answer(question, context):
    inputs = tokenizer(
        question,
        context,
        return_tensors="pt",
        max_length=max_length,
        truncation="only_second",
        padding="max_length",
    ).to(device)
    
    model.to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    
    answer_start = torch.argmax(outputs.start_logits)
    answer_end = torch.argmax(outputs.end_logits)
    
    if answer_start >= len(inputs["input_ids"][0]) or answer_end >= len(inputs["input_ids"][0]):
        return "No answer found"
    
    # Convert to numpy for easier manipulation
    input_ids = inputs["input_ids"][0].cpu().numpy()
    
    # Check if the answer is a no-answer prediction
    if answer_start == 0 and answer_end == 0:
        return "No answer found"
    
    answer = tokenizer.decode(input_ids[answer_start:answer_end+1], skip_special_tokens=True)
    return answer

In [ ]:
# Test the model with a few examples from the validation set
for i in range(5):  # Test 5 examples
    example = dataset["validation"][i]
    print(f"\nExample {i+1}:")
    print(f"Context: {example['context'][:200]}...")
    print(f"Question: {example['question']}")
    print(f"True answer: {example['answers']['text']}")
    predicted_answer = get_answer(example['question'], example['context'])
    print(f"Predicted answer: {predicted_answer}")

## Save the Model

In [ ]:
# Save the model and tokenizer
model_save_path = "./nasa-smd-qa-final"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)
print(f"Model saved to {model_save_path}")